Install ollama and pull model(s)

In [1]:
!pip install ollama
!ollama pull mistral
#!ollama pull llama3.1

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         
pulling 491dfa501e59... 100% ▕████████████████▏  801 B                         
pulling ed11eda7790d... 100% ▕████████████████▏   30 B                         
pulling 42347cd80dc8... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [2]:
!ollama serve # if ollama is already running, this will fail with the error "bind: address already in use"

Error: listen tcp 127.0.0.1:11434: bind: address already in use


Install Llama-Index

In [3]:
#%pip install llama-index --upgrade
%pip install llama-index-core --upgrade
%pip install llama-index-llms-ollama
%pip install llama-index-embeddings-huggingface
#%pip install llama-index-postprocessor-cohere-rerank
%pip install llama-index-postprocessor-flag-embedding-reranker
%pip install git+https://github.com/FlagOpen/FlagEmbedding.git
#%pip install llama-index-postprocessor-nvidia-rerank
%pip install llama-index-callbacks-arize-phoenix
%pip install llama-index-readers-file
%pip install llama-parse

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/FlagOpen/FlagEmbedding.git to /private/var/folders/gp/ff0gcwjx6z57vnqrd2ffs6r00000gn/T/pip-req-build-zibnbui9
  Running command git clone --filter=blob:none --quiet https://github.com/FlagOpen/FlagEmbedding.git /private/var/folders/gp/ff0gcwjx6z57vnqrd2ffs6r00000gn/T/pip-req-build-zibnbui9
  Resolved https://github.com/FlagOpen/FlagEmbedding.git to commit 274f4c055a3685e0296a1593fd822f63b98bbca9
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Install telemetry and visualization

In [4]:
%pip install opentelemetry-proto --upgrade
%pip install arize-phoenix-otel
%pip install pyvis

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Load env variables from .env

In [5]:
import os
from pathlib import Path
from dotenv import load_dotenv

dotenv_path = Path('../.env') # path to your .env file
load_dotenv(dotenv_path=dotenv_path)

DATA_PATH = os.getenv('DATA_PATH')
PERSIST_DIR = os.getenv('PERSIST_DIR')
PERSIST_RECURSIVE_DIR = os.getenv('PERSIST_RECURSIVE_DIR')
LLAMA_API_KEY = os.getenv('LLAMA_API_KEY')

In [6]:
from typing import Optional

from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks.base_handler import BaseCallbackHandler
from llama_index.core.callbacks.schema import CBEventType, EventPayload

class SubQueryLogger(BaseCallbackHandler):
    def __init__(self):
        super().__init__(
            event_starts_to_ignore=[],
            event_ends_to_ignore=[]
        )

    def start_trace(self, trace_id: str, **kwargs):
        # Optional: Implement any setup before tracing starts
        print(f"Starting trace with ID: {trace_id}")

    def end_trace(self, trace_id: str, **kwargs):
        # Optional: Implement any cleanup after tracing ends
        print(f"Ending trace with ID: {trace_id}")

    def on_event_start(
        self, event_type: str, payload: dict, event_id: str, parent_id: Optional[str] = None, **kwargs
    ):
        if event_type == CBEventType.QUERY:
            query_str = payload.get(EventPayload.QUERY_STR, '') if payload else 'NONE'
            print(f"\n---\nStarting Query: {query_str}\n---")
        elif event_type == CBEventType.SUB_QUESTION:
            print(f"\n---\nStarting SubQuery: {payload}\n---")
        elif event_type == CBEventType.LLM:
            messages = payload.get(EventPayload.MESSAGES, '')
            print("\nLLM Call with messages:\n")
            for i, m in enumerate(messages):
                 print(f"\n--- Message {i+1} ---\n{m}")
            print("\n")
        elif event_type == CBEventType.RETRIEVE:
            query_str = payload.get(EventPayload.QUERY_STR, '') if payload else 'NONE'
            print(f"\nRetrieve start:\n{query_str}\n")
        elif event_type == CBEventType.RERANKING:
            nodes = payload.get(EventPayload.NODES, '')
            print("\nReranking start:\n")
            for i, n in enumerate(nodes):
                 print(f"\n--- Node {i+1} ---\n{n}")
            print("\n")

    def on_event_end(
        self, event_type: str, payload: dict, event_id: str, parent_id: Optional[str] = None, **kwargs
    ):
        if event_type == CBEventType.QUERY:
            response = payload.get(EventPayload.RESPONSE, '') if payload else 'NONE'
            print(f"\n---\nEnding Query with Response: {response}\n---")
        elif event_type == CBEventType.SUB_QUESTION:
            print(f"\n---\nEnding SubQuery: {payload}\n---") if payload else 'NONE'
        elif event_type == CBEventType.LLM:
            response = payload.get(EventPayload.RESPONSE, '')
            print(f"\nLLM Response:\n{response}\n")
        elif event_type == CBEventType.RETRIEVE:
            nodes = payload.get(EventPayload.NODES, '')
            print("\nRetrieve end:")
            for i, n in enumerate(nodes):
                 print(f"\n--- Node {i+1} ---\n{n}")
                 print(n.get_content())
            print("\n")
        elif event_type == CBEventType.RERANKING:
            nodes = payload.get(EventPayload.NODES, '')
            print("\nReranking end:\n")
            for i, n in enumerate(nodes):
                 print(f"\n--- Node {i+1} ---\n{n}")
            print("\n")

# Initialize the callback manager
callback_manager = CallbackManager([SubQueryLogger()])

Configure LLama-Index

In [7]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = Ollama(model="mistral")
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2") #using for clustering and similarity search
Settings.callback_manager = callback_manager

/opt/miniconda3/envs/torch-mlx/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
from phoenix.otel import register

tracer_provider = register(
  project_name="default",
  endpoint="http://localhost:6006/v1/traces"
)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [9]:
import phoenix as px
session = px.launch_app()
print(session)
import llama_index.core

llama_index.core.set_global_handler("arize_phoenix")

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

if torch.backends.mps.is_available():
    print("MPS (Apple Metal) is available.")
    device ='mps'
else:
    print("MPS (Apple Metal) is not available.")
    device = 'cpu'

model_name = "mistralai/Mistral-7B-Instruct-v0.3" # for text generation use Mistral native tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

MPS (Apple Metal) is available.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Create CallbackManager for tracing 

In [11]:
import nest_asyncio
nest_asyncio.apply()

In [29]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

reranker = FlagEmbeddingReranker(
    top_n=2,
    model="BAAI/bge-reranker-large"
)

In [30]:
import os.path
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
from llama_parse import LlamaParse

if not os.path.exists(PERSIST_DIR):
    pdf_parser = LlamaParse(
        api_key=LLAMA_API_KEY,
        result_type='markdown', # "markdown" and "text" are available
        num_workers=4,  # if multiple files passed, split in `num_workers` API calls
        verbose=True,
        language="en",  # Optionally you can define a language, default=en
    )
    file_extractor = {".pdf": pdf_parser}
    documents = SimpleDirectoryReader(input_dir=DATA_PATH, 
    #                                 file_extractor=file_extractor
                                      ).load_data()
    index = VectorStoreIndex.from_documents(documents, callback_manager=callback_manager) # by default uses Settings.embed_model
    index.set_index_id("vector_index")
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)     
    index = load_index_from_storage(
        storage_context, 
        index_id="vector_index", 
        #callback_manager=callback_manager
    )             
         
query_engine = index.as_query_engine(
    similarity_top_k=5, 
    node_postprocessors=[reranker], 
    verbose=True
)


Starting trace with ID: index_construction
Ending trace with ID: index_construction


In [31]:
query = "Purchases of marketable securities in 2020"
response = query_engine.query(query)

Starting trace with ID: query

---
Starting Query: Purchases of marketable securities in 2020
---

Retrieve start:
Purchases of marketable securities in 2020


Retrieve end:

--- Node 1 ---
Node ID: b55b16b4-413e-4571-bc06-3ecdbfedc57e
Text: Treasury securities  28,439  331  —  28,770  8,580  11,972
8,218  U.S. agency securities  8,604  8  —  8,612  2,009  3,078  3,525
Non-U.S. government securities  19,361  275  (186)  19,450  255  3,329
15,866  Certificates of deposit and time  deposits  10,399  —  —
10,399  4,043  6,246  110  Commercial paper  11,226  —  —  11,226
3,185  8,0...
Score:  0.612

Treasury securities  28,439  331  —  28,770  8,580  11,972  8,218 
U.S. agency securities  8,604  8  —  8,612  2,009  3,078  3,525 
Non-U.S. government securities  19,361  275  (186)  19,450  255  3,329  15,866 
Certificates of deposit and time 
deposits  10,399  —  —  10,399  4,043  6,246  110 
Commercial paper  11,226  —  —  11,226  3,185  8,041  — 
Corporate debt securities  76,937  1,834  (

In [32]:
response

Response(response=' In the provided context, it is stated that as of September 26, 2020, the purchases (adjusted cost) of marketable securities were $8,604 million for U.S. agency securities and $28,439 million for U.S. Treasury securities. These amounts are listed under the "Level 1" investment category in the table titled "Cash, Cash Equivalents and Marketable Securities".', source_nodes=[NodeWithScore(node=TextNode(id_='b55b16b4-413e-4571-bc06-3ecdbfedc57e', embedding=None, metadata={'page_label': '41', 'file_name': 'apple_2021_10k.pdf', 'file_path': '/Users/ilyaivensky/workspace/RAG/data/raw/apple_2021_10k.pdf', 'file_type': 'application/pdf', 'file_size': 789896, 'creation_date': '2024-09-20', 'last_modified_date': '2021-10-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_dat

Recursive index and query engine

In [ ]:
from copy import deepcopy
from llama_index.core.schema import TextNode


def get_page_nodes(docs, separator="\n---\n"):
    """Split each document into page node, by separator."""
    nodes = []
    for doc in docs:
        doc_chunks = doc.text.split(separator)
        for doc_chunk in doc_chunks:
            node = TextNode(
                text=doc_chunk,
                metadata=deepcopy(doc.metadata),
            )
            nodes.append(node)

    return nodes

In [16]:
documents = LlamaParse(
    api_key=LLAMA_API_KEY, 
    result_type="markdown").load_data(Path(DATA_PATH) / "apple_2021_10k.pdf")

Started parsing the file under job_id 289183c5-54d2-4057-8af9-36e3242de76e


In [18]:
page_nodes = get_page_nodes(documents)
len(page_nodes)

82

In [19]:
from llama_index.core.node_parser import MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser(
    #llm=Settings.llm, 
    num_workers=4
)

nodes = node_parser.get_nodes_from_documents(documents)


2it [00:00, 29330.80it/s]


Starting trace with ID: index_construction
Ending trace with ID: index_construction
Starting trace with ID: query

---
Starting Query: What is this table about? Give a very concise summary (imagine you are adding a new caption and summary for this table), and output the real/existing table title/caption if context provided.and output the real/existing table id if context provided.and also output whether or not the table should be kept.
---

Retrieve start:
What is this table about? Give a very concise summary (imagine you are adding a new caption and summary for this table), and output the real/existing table title/caption if context provided.and output the real/existing table id if context provided.and also output whether or not the table should be kept.


Retrieve end:

--- Node 1 ---
Node ID: af24ea6f-ebe9-4a94-b946-4fe89cac8695
Text: |California|94-2404110| |---|---| |(State or other
jurisdiction|(I.R.S. Employer Identification No.)| |of incorporation
or organization)| |
Score: Non

1it [00:00, 25575.02it/s]

Starting trace with ID: index_construction
Ending trace with ID: index_construction
Starting trace with ID: query

---
Starting Query: What is this table about? Give a very concise summary (imagine you are adding a new caption and summary for this table), and output the real/existing table title/caption if context provided.and output the real/existing table id if context provided.and also output whether or not the table should be kept.
---

Retrieve start:
What is this table about? Give a very concise summary (imagine you are adding a new caption and summary for this table), and output the real/existing table title/caption if context provided.and output the real/existing table id if context provided.and also output whether or not the table should be kept.


Retrieve end:

--- Node 1 ---
Node ID: d66c1525-5711-44e6-9dc5-1be75b5709df
Text: |Large accelerated filer|☒|Accelerated filer|☐|
|---|---|---|---| |Non-accelerated filer|☐|Smaller reporting
company|☐| |Emerging growth company|☐| | 


LLM Response:
assistant:  Caption: Filing Status Classification Table
   Summary: This table provides a classification of companies based on their filing status, separating them into accelerated filers and non-accelerated filers. The column 'Large Accelerated Filer' denotes those who file their financial reports ahead of schedule, while the 'Non-accelerated filer' column indicates those who do not. The table does not contain any information about smaller reporting companies or emerging growth companies.
   Table Title (if provided): None (The context did not provide a title for this table)
   Table ID (if provided): None (The context did not provide an ID for this table)
   Keep the table: Yes, it provides useful information on company filing statuses.


---
Ending Query with Response: NONE
---
Ending trace with ID: query
Starting trace with ID: query

---
Starting Query: What is this table about? Give a very concise summary (imagine you are adding a new caption and summary for this t

...

Starting trace with ID: index_construction
Ending trace with ID: index_construction
Starting trace with ID: query

---
Starting Query: What is this table about? Give a very concise summary (imagine you are adding a new caption and summary for this table), and output the real/existing table title/caption if context provided.and output the real/existing table id if context provided.and also output whether or not the table should be kept.
---

Retrieve start:
What is this table about? Give a very concise summary (imagine you are adding a new caption and summary for this table), and output the real/existing table title/caption if context provided.and output the real/existing table id if context provided.and also output whether or not the table should be kept.


Retrieve end:

--- Node 1 ---
Node ID: 84bfd6e3-e34f-4898-a487-ad54cff2c45c
Text: |Name|Title|Date| |---|---|---| |/s/ Timothy D. Cook|Chief
Executive Officer and Director (Principal Executive Officer)|October
28, 2021| |/s/ Luca Ma


LLM Response:
assistant:  Title: Board of Directors and Executive Officers at a Company (as of October 28, 2021)
   Summary: This table provides an overview of the current board members and key executives within a company as of October 28, 2021. The information includes their names, titles, and dates of appointment.
   Keep this table.


---
Ending Query with Response: NONE
---
Ending trace with ID: query
Starting trace with ID: query

---
Starting Query: What is this table about? Give a very concise summary (imagine you are adding a new caption and summary for this table), and output the real/existing table title/caption if context provided.and output the real/existing table id if context provided.and also output whether or not the table should be kept.
---

Retrieve start:
What is this table about? Give a very concise summary (imagine you are adding a new caption and summary for this table), and output the real/existing table title/caption if context provided.and output the real/exi

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 41527.76it/s]


Starting trace with ID: index_construction
Ending trace with ID: index_construction
Starting trace with ID: query

---
Starting Query: What is this table about? Give a very concise summary (imagine you are adding a new caption and summary for this table), and output the real/existing table title/caption if context provided.and output the real/existing table id if context provided.and also output whether or not the table should be kept.
---

Retrieve start:
What is this table about? Give a very concise summary (imagine you are adding a new caption and summary for this table), and output the real/existing table title/caption if context provided.and output the real/existing table id if context provided.and also output whether or not the table should be kept.


Retrieve end:

--- Node 1 ---
Node ID: 81488f3f-769a-4469-82a2-419bc4f6350a
Text: |Subsidiary|Jurisdiction of Incorporation| |---|---| |Apple Asia
Limited|Hong Kong| |Apple Asia LLC|Delaware, U.S.| |Apple Canada
Inc.|Canada| |Apple 

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [20]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)


In [21]:
len(base_nodes), len(objects)

(146, 60)

In [22]:
# dump both indexed tables and page text into the vector index
recursive_index = VectorStoreIndex(nodes=base_nodes + objects + page_nodes)
recursive_index.set_index_id("vector_index")
recursive_index.storage_context.persist(persist_dir=PERSIST_RECURSIVE_DIR)

Starting trace with ID: index_construction
Ending trace with ID: index_construction


In [23]:
storage_context_recursive = StorageContext.from_defaults(persist_dir=PERSIST_RECURSIVE_DIR)     
recursive_index = load_index_from_storage(
    storage_context_recursive, 
    index_id="vector_index")     

Starting trace with ID: index_construction
Ending trace with ID: index_construction


In [24]:

recursive_query_engine = recursive_index.as_query_engine(    
    similarity_top_k=5, 
    node_postprocessors=[reranker]
)


In [25]:
recursive_response = recursive_query_engine.query(query)

Starting trace with ID: query

---
Starting Query: Purchases of marketable securities in 2020
---

Retrieve start:
Purchases of marketable securities in 2020


Retrieve end:

--- Node 1 ---
Node ID: d5e2ca3f-8267-4720-afc2-28564cfd937f
Text: PART II   Item 5. Market for Registrant’s Common Equity, Related
Stockholder Matters and Issuer Purchases of Equity Securities  The
Company’s common stock is traded on The Nasdaq Stock Market LLC under
the symbol AAPL.   Holders  As of October 15, 2021, there were 23,502
shareholders of record.   Purchases of Equity Securities by the Issuer
and ...
Score:  0.524

PART II

 Item 5. Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities

The Company’s common stock is traded on The Nasdaq Stock Market LLC under the symbol AAPL.

 Holders

As of October 15, 2021, there were 23,502 shareholders of record.

 Purchases of Equity Securities by the Issuer and Affiliated Purchasers

Share repurchase activ

In [26]:
query

'Purchases of marketable securities in 2020'

In [27]:
recursive_response

Response(response=" In 2020, the Company entered into agreements to sell certain of its marketable securities with a promise to repurchase them at a specified time and amount, which were accounted for as collateralized borrowings. However, no specific information about the purchases of marketable securities in 2020 was provided in the context you've given. The discussion revolves around share repurchases of their own common stock.", source_nodes=[NodeWithScore(node=TextNode(id_='29bd767b-34aa-4300-86d0-17f345c8419c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='cffe9347-eda8-428c-835c-a3ef09f660c9', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='7233075df8434a2befadd1ba07b3f36210f1c83c4a750e4474c393169583ddaa'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='d5807a89-0213-4868-99db-2e36f9c5778c', node_type=<ObjectType.TEXT: '1'>, metadata={'table

In [28]:
print(recursive_response.source_nodes[0].get_content())

In 2020, the Company entered into agreements to sell certain of its marketable securities with a promise to repurchase the securities at a specified time and amount (“Repos”).

Due to the Company’s continuing involvement with the marketable securities, the Company accounted for its Repos as collateralized borrowings.

The Company entered into $5.2 billion of Repos during 2020, all of which had been settled as of September 26, 2020.

Apple Inc. | 2021 Form 10-K | 44
